<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/Demo_AdiabaticFlash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries
This section is somewhat intimidating to new users, but modifications are necessary only to access advanced features.

In [1]:
!git clone --depth 1 https://github.com/profteachkids/CHE2064.git &> /dev/null
!pip install DotMap &> /dev/null
import sys
sys.path.insert(1, "/content/CHE2064") #Path to CHE module imports

In [1]:
from dotmap import DotMap
import pandas as pd
import jax
import jax.numpy as jnp
from jax.config import config
config.update("jax_enable_x64", True) #JAX default is 32bit single precision

from tools.tree_array_transform import VSC, Comp, Range
import tools.che as che

## Dot-access to chemical properties and calculations
ChemCAD data properties and binary interaction parameters can be exported as text files.  A few common chemicals are stored in a Github repository.  Users with ChemCad access can export and access their own data files.  The *che.Props* class parses these files for convenient dot-access and implementations of common calculations.

In [2]:
p = che.Props(['Ethanol','Isopropanol', 'Water'])

In [3]:
print(f'Tc: {p.Tc}')
print(f'Pc: {p.Tc}')
print(f'Vapor Pressure at 432.1 K {p.Pvap(432.1)} Pa')
print(f'NRTL activity coefficients for equimolar mixture {p.NRTL_gamma([1/3,1/3,1/3], 300)} ')

Tc: [513.92 508.31 647.35]
Pc: [513.92 508.31 647.35]
Vapor Pressure at 432.1 K [1220670.06610508 1069634.05615236  600793.6517417 ] Pa
NRTL activity coefficients for equimolar mixture [1.11669551 1.12519412 1.85757473] 


/Users/carlosco/opt/anaconda3/envs/pytorch/lib/python3.8/site-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


## Adiabatic Flash Calculation
Model mass/energy balance and VLE - all variables (knowns and unknowns) are in a convenient DotMap structure (c).  The model is written once.

Sum up weighted square deviations for minimization.
 

In [10]:
def model(c,r):
    # c: combination of adjustable variables and static state parameters
    # r: DotMap - store intermediate results for reporting
    Ltot = c.Ftot - c.Vtot  # What's not in the vapor is in the liquid
    r.V = c.Vy * c.Vtot # Moles of each component = mole fractions * total moles
    r.L = c.Lx * Ltot
    r.F = c.Fz * c.Ftot
    mass_balance = (r.F - r.V - r.L) # Mass balance for each component (vectors!)
    mass_balance = (mass_balance/jnp.sum(r.F))**2

    # Hmix calculates the enthalpy given the temperature and moles of each
    # component in the vapor and liquid phases
    r.FH = p.Hl(nL=r.F, T=c.FT)
    r.VH = p.Hv(nV=r.V, T=c.flashT)
    r.LH = p.Hl(nL=r.L, T=c.flashT)
    energy_balance = (r.FH - r.VH - r.LH)
    energy_balance = (energy_balance/jnp.sum(r.FH))**2

    # Raoults with NRTL activity coefficient correction.  One-liner!
    VLE = (c.Lx  * p.NRTL_gamma(c.Lx,c.flashT)* p.Pvap(c.flashT)
                - c.Vy*c.flashP)
    VLE = (VLE/jnp.sum(c.Vy*c.flashP))**2

    # Square deviations are weighted by their approx. magnitudes and
    # summed for minimization
    return jnp.sum(mass_balance + energy_balance + VLE)

## What's known (static) and unknown (variable) - automatically combined and tracked.

What's known and unknown can be easily swapped around without having to modify the model.  The DotMap structures can have nested lists of DotMaps with nested lists of arrays, and so forth.

In [11]:
# Static parameters (Total feed, feed mole fractions, feed temperature and )
s=DotMap()
s.Ftot=10 # Total Feed moes
s.Fz = jnp.array([1/3, 1/3, 1/3]) # Equimolar feed composition
s.FT = 450 # Feed temperature
s.flashP= 101325 # Flash drum pressure

# Variables (to be solved for with initial guesses and bounds)
v=DotMap()
v.Vy = Comp(s.Fz) # Guess vapor/liquid composition equal to feed
v.Lx = Comp(s.Fz) # Comp - constrains mole fractions to behave like mole fractions!
v.flashT = Range(360, 273.15, s.FT)  # Guess and bounds for flash temperature 
v.Vtot = Range(s.Ftot/2, 0., s.Ftot)  # Guess half of feed in vapor

## Magic happens here
The VSC class combines unknown variables and static parameters into a convenient DotMap structure.  Transformations between known/unknown DotMaps and the flat arrays required by the minimization routine is automated.

The minimization algorithm is a robust Trust-Newton Conjugate Gradient coded to take advantage of JAX automatic Jacobian vector products.  

The initial JAX model compilation is a bit slow, but subsequent repeat calculations are fast.

In [13]:
vsc=VSC(v,s, model)
vsc.minimize(jit=False, verbosity=1)

0.06224976735089524
0.006248548166419529
0.0018775481930001286
0.0018775481930001286
0.0018775481930001286
1.1689747101334969e-05
1.1689747101334969e-05
1.086438266234864e-06
1.086438266234864e-06
2.9661262250309602e-08
2.9661262250309602e-08
8.365011152459128e-10
8.365011152459128e-10
2.4159135162884922e-11
2.4159135162884922e-11
2.4159135162884922e-11
7.846645257675808e-13
7.846645257675808e-13
2.6875660370326527e-14
2.6875660370326527e-14
1.0869792501504432e-15
4.424887863842934e-17


vector1   vector3                    
             1         1         2         3
Ftot        10                              
Fz              0.333333  0.333333  0.333333
FT         450                              
flashP  101325

In [15]:
# State parameters
vsc.sdf

vector1   vector3                    
             1         1         2         3
Ftot        10                              
Fz              0.333333  0.333333  0.333333
FT         450                              
flashP  101325

In [16]:
# Adjustable Variables
vsc.vdf

vector3                      vector1
               1         2         3        1
Lx      0.311418  0.326803  0.361779         
Vtot                                   3.9739
Vy      0.366567  0.343236  0.290198         
flashT                                352.966

In [17]:
# Intermediate results
vsc.rdf



vector3                        vector1
          1        2        3            1
V    1.4567  1.36398  1.15322             
L   1.87664  1.96935  2.18012             
F   3.33333  3.33333  3.33333             
FH                             6.37096e+08
VH                             3.49014e+08
LH                             2.88082e+08